In [10]:
from sqlalchemy import URL, create_engine
import os

from dotenv import load_dotenv
load_dotenv(dotenv_path="./boot/db_init/db.env")
load_dotenv(dotenv_path="./dev.env", override=True)

from client.sql_models import Base, LogsBase

def read_pwd_file(file_path:str) -> str:
    with open(file=file_path, mode="r") as pwd_file:
        pwd = pwd_file.readline().replace("\n", "")
    return pwd

connect_args = {
    "ssl": {
        "ca"                : os.getenv("CLIENT_CERTIF_PATH") + "ca-cert.pem",
        "cert"              : os.getenv("CLIENT_CERTIF_PATH") + "client-cert.pem",
        "key"               : os.getenv("CLIENT_CERTIF_PATH") + "client-key.pem",
        "check_hostname"    : False,
    }
}

sql_logs_url = URL.create(
    drivername="mariadb+pymysql",
    username=read_pwd_file("./boot/db_init/mariadb_log_user_name"),
    password=read_pwd_file("./boot/db_init/mariadb_log_user_pwd"),
    host="localhost",
    port=3306,
    database=os.getenv("MARIADB_FOR_LOGS"),
)
logs_engine = create_engine(
    url=sql_logs_url
    , connect_args=connect_args
)
LogsBase.metadata.create_all(logs_engine)

In [46]:
from __future__ import annotations
from typing import List
from sqlalchemy import ForeignKey, Table, Column, UniqueConstraint
from sqlalchemy import String, DateTime, Integer, func, DATETIME
from sqlalchemy.types import BLOB
from sqlalchemy.ext.asyncio import AsyncAttrs
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship, WriteOnlyMapped
from datetime import datetime


class Base(DeclarativeBase, AsyncAttrs):
    pass

role_association_table = Table(
    "role_association_table"
    , Base.metadata
    , Column("user_id", ForeignKey("user_account.id"))
    , Column("role_id", ForeignKey("role.id"))
)

class User(Base):
    __tablename__ = "user_account"
    
    id                  :   Mapped[int] = mapped_column(primary_key=True)
    email               :   Mapped[str] = mapped_column(String(264))
    password            :   Mapped[str] = mapped_column(String(264))
    firstname           :   Mapped[str | None] = mapped_column(String(30))
    lastname            :   Mapped[str | None] = mapped_column(String(30))
    role                :   Mapped[List[Role]] = relationship(secondary=role_association_table)
    create_date         :   Mapped[datetime] = mapped_column(insert_default=func.now())
    desactivation_date  :   Mapped[datetime | None] = mapped_column(DATETIME())
    search              :   WriteOnlyMapped["SearchResults"] = relationship(
                                                                                    cascade="all, delete-orphan"
                                                                                    , passive_deletes=True
                                                                                    , order_by="SearchResults.date_of_search")

    __table_args__ = (UniqueConstraint("email"), )

    @property
    def scopes(self) -> set[str] | None:
        if self.role :
            set_out = set()
            for role in self.role:
                set_out.add(role.scope)
            return set_out
        else :
            return None
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.email!r}, fullname={self.firstname!r} {self.lastname!r})"


class Role(Base):
    __tablename__ = "role"
    id              : Mapped[int] = mapped_column(primary_key=True)
    scope           : Mapped[str] = mapped_column(String(30))
    description     : Mapped[str] = mapped_column(String(264))

class SearchResults(Base):
    __tablename__ = "search"
    id                      :   Mapped[int] = mapped_column(primary_key=True)
    search_index            :   Mapped[str] = mapped_column(String(250))
    date_of_search          :   Mapped[datetime] = mapped_column(DateTime())
    search_type             :   Mapped[str] = mapped_column(String(3)) ### WEB or API
    search_platform         :   Mapped[str] = mapped_column(String(48))
    user_id                 :   Mapped[int | None] = mapped_column(ForeignKey("user_account.id", ondelete="cascade"))
    generated_paragraphs    :   Mapped[List['GeneratedParagraphs']] = relationship(back_populates="search")


class GeneratedParagraphs(Base):
    __tablename__ = "generated_paragaphs"
    id                          :   Mapped[int] = mapped_column(primary_key=True)
    generated_pargraphs_es_id   :   Mapped[str] = mapped_column(String(250))
    noted                       :   Mapped[int | None] = mapped_column(Integer())
    search_id                   :   Mapped[int] = mapped_column(ForeignKey("search.id"))
    search                      :   Mapped['SearchResults'] = relationship(back_populates="generated_paragraphs")

    

In [34]:
from sqlalchemy import URL
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker
import os
import ssl

from dotenv import load_dotenv
load_dotenv(dotenv_path="./boot/db_init/db.env")
load_dotenv(dotenv_path="./dev.env", override=True)

from client.sql_models import Base, LogsBase

def read_pwd_file(file_path:str) -> str:
    with open(file=file_path, mode="r") as pwd_file:
        pwd = pwd_file.readline().replace("\n", "")
    return pwd

ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
ssl_context.load_verify_locations(os.getenv("CLIENT_CERTIF_PATH") + "ca-cert.pem")
ssl_context.load_cert_chain(certfile=os.getenv("CLIENT_CERTIF_PATH") + "client-cert.pem", keyfile=os.getenv("CLIENT_CERTIF_PATH") + "client-key.pem",)
ssl_context.check_hostname = False



connect_args = {
    "ssl": ssl_context
    # {
    #     "ca"                : os.getenv("CLIENT_CERTIF_PATH") + "ca-cert.pem",
    #     "cert"              : os.getenv("CLIENT_CERTIF_PATH") + "client-cert.pem",
    #     "key"               : os.getenv("CLIENT_CERTIF_PATH") + "client-key.pem",
    #     "check_hostname"    : False,
    # }
}

sql_logs_url = URL.create(
    drivername="mariadb+aiomysql",
    username=os.getenv("MARIADB_USER"),
    password=read_pwd_file("./boot/db_init/mariadb_mysql_pwd.txt"),
    host="localhost",
    port=3306,
    database=os.getenv("MARIADB_DATABASE"),
)
async_engine = create_async_engine(
    url=sql_logs_url
    , connect_args=connect_args
)
async with async_engine.begin() as conn:
    await conn.run_sync(Base.metadata.create_all)
# await Base.metadata.create_all(logs_engine)

In [40]:
from typing import Annotated
from pydantic import SkipValidation, BaseModel, Field, validator, computed_field, ValidationError, ValidationInfo, field_validator, ConfigDict, model_validator
from datetime import datetime
import os

class Token(BaseModel):
    access_token    : str
    token_type      : str

class TokenData(BaseModel):
    username    : str | None = None
    scopes      : list[str] = []


class UserBase(BaseModel):
    email       : str
    firstname   : str | None = None
    lastname    : str | None = None

    @field_validator('email')
    @classmethod
    def check_email(cls, v : str, info: ValidationInfo) -> str:
        if isinstance(v, str):
            is_email = "@" in v
            assert is_email, f"{info.field_name} must contain a '@'"
        return v
    

class UserCreate(UserBase):
    password : str

class UserOut(UserBase):
    id                  : int
    scopes                : Annotated[set[str], Field(validate_default=True)]
    create_date         : datetime
    desactivation_date  : datetime | None = None

    # @field_validator('role', mode='before')
    # @classmethod
    # async def get_awaitable_elements(cls, v, info: ValidationInfo) -> str:
    #     value_to_return = await v.awaitable_attrs.scopes
    #     return value_to_return

class UserMod(UserOut):
    model_config = ConfigDict(from_attributes=True)


class UserDeleted(UserOut):
    model_config = ConfigDict(from_attributes=True)
    deleted             : bool = True


# class GeneratedParagraph(BaseModel):
#     id                          : int
#     generated_pargraphs_es_id   : str
#     noted                       : int

class SummerizedParagraph(BaseModel):
    id          : str
    title       : str
    content     : str
    refs        : list[str]

    # class Config:
    #     orm_mode = True

class UserInDB(UserMod):
    hashed_password: str

class SearchBase(BaseModel):
    search_platform         : str

class SearchOneRetrieve(SearchBase):
    id                      : int
    search_index            : str
    search_type             : str


class SearchDeleted(SearchOneRetrieve):
    user_id                 : int
    deleted                 : bool = True

    class Config:
        orm_mode = True

class Search(SearchOneRetrieve):
    user_id                 : int
    date_of_search          : datetime
    generated_paragraphs    : list[SummerizedParagraph]

class SearchRequest(SearchBase):
    prompt          : str
    search_type     : str = "api"

class TaskBase(BaseModel):
    id      : str
    name    : str

class TaskCreated(TaskBase):
    send_time       : datetime = datetime.now()

    @computed_field
    def retrieve_result_url(self) -> str :
        return os.getenv('API_ENDPOINT') + "/tasks/" + self.id

    # retrive_url     : str = Field()

    class Config:
        orm_mode = True


class TaskResult(TaskBase):
    status  : str
    result  : Search | None = None

    class Config:
        orm_mode = True






/Users/etiennewagner/miniconda3/envs/projet_certif/lib/python3.12/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [56]:
from sqlalchemy import select
from sqlalchemy.orm import subqueryload, selectinload, joinedload

SessionLocal = async_sessionmaker(autoflush=False, bind=async_engine)
async with SessionLocal() as db :
    
    users = await db.scalars(
        select(User).options(joinedload(User.role))
            .where(User.email == "etienne.wagner@gmail.com")
            .limit(1)
    )
    # user = UserMod.from_orm(users.one())
    user = users.unique().one_or_none()
    # UserMod.model_validate(user)
    # scopes : list[Role] = await user.awaitable_attrs.role
    # for scope in await user.awaitable_attrs.scopes :
    #     print(scope)
await db.close_all()

/var/folders/yp/9gm2w_f53xj6vsx7dgtz95_c0000gn/T/ipykernel_25525/2401427262.py:18: SADeprecationWarning: The AsyncSession.close_all() method is deprecated and will be removed in a future release.  Please refer to close_all_sessions(). (deprecated since: 2.0)
  await db.close_all()


In [57]:
UserMod.model_validate(user)

UserMod(email='etienne.wagner@gmail.com', firstname=None, lastname=None, id=1, scopes={'admin', 'me'}, create_date=datetime.datetime(2024, 5, 13, 13, 37, 23), desactivation_date=None)

In [88]:
user

User(id=1, name='etienne.wagner@gmail.com', fullname=None None)

In [76]:
scopes[0].

'me'

In [64]:
scopes = await user.awaitable_attrs.scopes

DetachedInstanceError: Parent instance <User at 0x11f3b8710> is not bound to a Session; lazy load operation of attribute 'role' cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)